<a href="https://colab.research.google.com/github/Eduardohg94/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Eduardo_Hugues_AG03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Nombre: Eduardo Hugues

#URL: https://colab.research.google.com/drive/13MJP6pK_z_7T2e3_tXrW0rF3MNZL_hGB

#URL: 

In [0]:
!pip install request

In [161]:
import urllib.request
file = 'swiss42.tsp'

urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp',file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7fed9d322b38>)

In [0]:
!pip install tsplib95

In [0]:
import tsplib95
import random
from math import e

In [162]:
problem = tsplib95.load_problem(file)

Nodos = list(problem.get_nodes())

list(problem.get_nodes())
list(problem.get_edges())

problem.wfunc(0,3)

23

In [0]:
def distancia(a,b,problem):
  return problem.wfunc(a,b)

def crear_solucion(Nodos): 
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion 

def distancia_total(solucion, problem):
  distancia_ret = 0
  for i in range(len(solucion)-1):
    distancia_ret += distancia(solucion[i],solucion[i+1],problem) 
  return distancia_ret + distancia(solucion[len(solucion)-1],solucion[0],problem)
    
solucion_prueba = crear_solucion(Nodos)
print(distancia_total(solucion_prueba,problem),solucion_prueba)

In [0]:
# busqueda aleatoria

def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = 10e10
  
  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia_solucion = distancia_total(solucion, problem)
  
  if distancia_solucion < mejor_distancia:
    mejor_solucion = solucion
    mejor_distancia = distancia_solucion
    
  print("La mejor solución encontrada es ", end="")
  print(solucion)
  print("con una distancia total de ", end="")
  print(mejor_distancia)
  return mejor_solucion

busqueda_aleatoria(problem,5000)

In [0]:
#busqueda local, generador de vecinos

In [0]:
def generar_vecina(solucion,problem):
  mejor_solucion = []
  mejor_distancia = 10e10
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i]+[solucion[j]] + solucion[i+1:j]+[solucion[i]]+solucion[j+1:]
      distancia_vecina = distancia_total(vecina,problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion

In [0]:
def busqueda_local(problem,N,inicios=1):
  mejor_solucion = []
  mejor_distancia = 10e10
  
  Nodos = list(problem.get_nodes())
  
  solucion_referencia = crear_solucion(Nodos)
  
#Se agrega un loop en el cual el algoritmo encuentra otro estado inicial aleatorio buscando así evitar terminar con un mínimo local como solución
  for j in range(0,inicios):
    for i in range(N):
      vecina = generar_vecina(solucion_referencia,problem)
      distancia_vecina = distancia_total(vecina,problem)
      if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
   
      solucion_referencia = mejor_solucion
  
  print(distancia_total(mejor_solucion,problem))
  return mejor_solucion

busqueda_local(problem,10,3)

In [0]:
def probabilidad(T,d):
  r=random.random()
  
  if r >= (e**(-1*d)/((T*5*18**(-5)))):
    return True
  else:
    return False
def bajar_temperatura(T):
  return T*.9

In [149]:
def recocido_simulado(problem,TEMPERATURA):
  solucion_referencia = crear_solucion(Nodos)
  
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  while TEMPERATURA >1:
    vecina = generar_vecina(solucion_referencia,problem)
    distancia_vecina = distancia_total(vecina,problem)
    
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
      
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
    
    return(mejor_distancia)
    
recocido_simulado(problem,1000)      

4913

In [0]:
def Add_Nodo(problem,H,T):
  #Mejora:Establecer una función de probabilidad para añadir un nuevo nodo dependiendo
  #de los nodos más cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(list(set(range(1,len(Nodos)))-set(H)))

def Incrementa_Feromona(problem,T,H):
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1008/distancia_total(H,problem)
  return T

def Evaporar_Feromonas(T):
  #Mejora: Podemos elegir diferentes funciones de evaporación según 
  T = [[max(T[i][j] - 0.3,1) for i in range(len(Nodos))] for j in range(len(Nodos))]
  return(T)

In [163]:
def hormigas(problem,N):
  Nodos = list(problem.get_nodes())
  Aristas = list(problem.get_edges())
  
  #Inicializar con valores diferentes según distintos criterios
  T = [[ 1 for i in range (len(Nodos))]for i in range(len(Nodos))]
  
  Hormiga = [[0] for i in range(N)]
  
  for h in range(N):
    for i in range(len(Nodos)-1):
      Nuevo_Nodo = Add_Nodo(problem,Hormiga[h],T)
    T = Incrementa_Feromona(problem, T, Hormiga[h])
    
    T = Evaporar_Feromonas(T)
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N):
    distancia_actual = distancia_total(Hormiga[h],problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia = distancia_actual
  
  return(mejor_solucion)    
hormigas(problem,100)

[0]